# Cloud Run Management
- create run with remote data
- manage runs via run.yaml
- serverless runtime

## Create run with remote data

### Configure ml_client

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<SUBSCRIPTION_ID>",
        "resource_group": "<RESOURCE_GROUP>",
        "workspace_name": "<AML_WORKSPACE_NAME>",
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: D:\code\prompt-flow\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000018F40ED67C0>,
         subscription_id=96aede12-2f73-41cb-b983-6d11a904839b,
         resource_group_name=promptflow,
         workspace_name=promptflow-eastus)


### Create or update remote data

In [4]:
from azure.ai.ml.entities import Data
from azure.core.exceptions import ResourceNotFoundError

data_name, data_version = "flow_run_test_data", "1"

try:
    data = ml_client.data.get(name=data_name, version=data_version)
except ResourceNotFoundError:
    data = Data(name=data_name, version=data_version, path=f"../../flows/standard/web-classification/data.jsonl", type="uri_file")
    data = ml_client.data.create_or_update(data)

### Prepare remote data id

In [5]:
data_id = f"azureml:{data.name}:{data.version}"
print(data_id)

azureml:flow_run_test_data:1


### Create a flow run with remote data

In [7]:
import promptflow.azure as pf

# configure global setting pointing to workspace ml_client
pf.configure(ml_client)

# create run
base_run = pf.run(
    # local flow file
    flow="../../flows/standard/web-classification",
    # remote data
    data=data_id,
    runtime="demo-mir",
)

Portal url: https://int.ml.azure.com/prompts/flow/bulkrun/run/web_classification_default_20230728_174553_495007/details?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-eastus&flight=promptfilestorage,PFSourceRun=false


### Stream the flow run to make sure it runs successfully

In [11]:
from promptflow.azure import PFClient

pf_client = PFClient(ml_client)
pf_client.runs.stream(base_run)

2023-07-28 09:46:07 +0000     214 promptflow-runtime INFO     [web_classification_default_20230728_174553_495007] Receiving v2 bulk run request a516ac28-05af-4f49-b3c6-65af0e0d9ac0: {"flow_id": "web_classification_default_20230728_174553_495007", "flow_run_id": "web_classification_default_20230728_174553_495007", "flow_source": {"flow_source_type": 1, "flow_source_info": {"snapshot_id": "c0b539cd-5975-4c2e-91f5-ce48028e5775"}, "flow_dag_file": "flow.dag.yaml"}, "connections": "**data_scrubbed**", "log_path": "https://promptfloweast4063704120.blob.core.windows.net/azureml/PromptflowLogs/web_classification_default_20230728_174553_495007.txt?sv=2021-10-04&se=2023-08-04T09%3A46%3A06Z&sr=b&sp=rw&sig=**data_scrubbed** "app_insights_instrumentation_key": "InstrumentationKey=**data_scrubbed**;IngestionEndpoint=https://eastus-6.in.applicationinsights.azure.com/;LiveEndpoint=https://eastus.livediagnostics.monitor.azure.com/", "data_inputs": {"data": "azureml:/subscriptions/96aede12-2f73-41cb-b98